In [58]:
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split,cross_validate,StratifiedKFold
from sklearn import svm
from keras.utils import to_categorical

In [59]:
X=pd.read_csv("data/X.csv", sep=' ',header=None, dtype=float)
X=X.values
Y=pd.read_csv("data/y_bush_vs_others.csv", header=None)
y = Y.values.ravel()
print(X.shape)
print(np.sum(y))

(13233, 4096)
530


In [60]:
print(X)
print(y)
y_cat = to_categorical(y)
classes = np.unique(y)
nClasses = len(classes)
print(nClasses)
print(y_cat)

[[0.10635443 0.09975202 0.11191927 ... 0.04983379 0.05402631 0.04627987]
 [0.         0.         0.         ... 0.00410788 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.00074521 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.00030042 0.         0.        ]]
[0 0 0 ... 0 0 0]
2
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [61]:
X_b = X.reshape(-1,64,64,1)
X_train, X_test, y_train, y_test = train_test_split(X_b, y, shuffle = True,stratify=y, test_size=1/3, random_state=7392)

In [62]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(8822, 64, 64, 1)
(4411, 64, 64, 1)
(8822,)
(4411,)


In [64]:
from keras import layers
from keras import models
from keras.layers.core import Activation
model = models.Sequential()
model.add(layers.Conv2D(64,(3,3),activation='relu',input_shape=(64,64,1)))
model.add(layers.MaxPooling2D(pool_size=(2, 2),strides=(2,2)))
model.add(layers.Conv2D(64, (3,3),activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2),strides=(2,2)))
model.add(layers.Conv2D(128, (3,3),activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2),strides=(2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 62, 62, 64)        640       
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 29, 29, 64)        36928     
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 12, 12, 128)       73856     
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 6, 6, 128)         0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 4608)              0         
__________

In [65]:
model.fit(X_train,y_train,batch_size=64,epochs=10)

Epoch 1/10
8822/8822 [==============================] - 86s 10ms/step - loss: 0.1852 - acc: 0.9559
Epoch 2/10
8822/8822 [==============================] - 86s 10ms/step - loss: 0.1509 - acc: 0.9599
Epoch 3/10
8822/8822 [==============================] - 87s 10ms/step - loss: 0.1295 - acc: 0.9611
Epoch 4/10
8822/8822 [==============================] - 86s 10ms/step - loss: 0.0916 - acc: 0.9698
Epoch 5/10
8822/8822 [==============================] - 88s 10ms/step - loss: 0.0637 - acc: 0.9787
Epoch 6/10
8822/8822 [==============================] - 85s 10ms/step - loss: 0.0471 - acc: 0.9824
Epoch 7/10
8822/8822 [==============================] - 89s 10ms/step - loss: 0.0347 - acc: 0.9886
Epoch 8/10
8822/8822 [==============================] - 87s 10ms/step - loss: 0.0288 - acc: 0.9905
Epoch 9/10
8822/8822 [==============================] - 82s 9ms/step - loss: 0.0264 - acc: 0.9910
Epoch 10/10
8822/8822 [==============================] - 82s 9ms/step - loss: 0.0148 - acc: 0.9956


In [66]:
scores = model.evaluate(X_test, y_test)

4411/4411 [==============================] - 12s 3ms/step


In [67]:
print(scores)

[0.04478308883475518, 0.9863976422579914]


In [68]:
predict_function= model.predict_classes(X_test)
print(predict_function.shape)
print(y_test.shape)

(4411, 1)
(4411,)


In [69]:
from sklearn.metrics import f1_score
f1_bush=f1_score(y_test,predict_function)
print(f1_bush)

0.8275862068965518


In [70]:
predict_function= model.predict_classes(X_train)
print(predict_function.shape)
print(y_train.shape)

(8822, 1)
(8822,)


In [71]:
from sklearn.metrics import f1_score
f1_bush=f1_score(y_train,predict_function)
print(f1_bush)

0.9843971631205675


In [72]:
model.save('bush.model')

In [73]:
from keras.models import load_model
model_final = load_model('bush.model')
model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 62, 62, 64)        640       
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 29, 29, 64)        36928     
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 12, 12, 128)       73856     
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 6, 6, 128)         0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 4608)              0         
__________

In [74]:
predict_function1 = model_final.predict_classes(train_X)
predict_function = model_final.predict_classes(test_X)
from sklearn.metrics import f1_score
f1_bush = f1_score(y_test,predict_function)
f1_bush1 = f1_score(y_train,predict_function1)
print(f1_bush1)
print(f1_bush)

0.9843971631205675
0.8275862068965518


In [75]:
import pickle

In [76]:
p = [f1_bush1, f1_bush]
print (p)

[0.9843971631205675, 0.8275862068965518]


In [77]:
pickle.dump(p, open('bush.pkl', 'wb'))